# Multiple linear regression

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import scipy.linalg as sp_la

## Data

Today we will keep working with the set of Craigslist listings for used cars.

In addition to the numeric columns odometer, year and price, I want to load the non-numeric columns manufacturer, model, condition, title_status, transmission and drive.

Because our numpy arrays all have to be the same type, we need to:
* figure out which columns we want
* define some converters

In [46]:
# these will be our columns
columns = ["Rented Bike Count","Hour","Temperature(C)","Humidity(%)","Wind speed (m/s)","Visibility (10m)","Dew point temperature(C)","Solar Radiation (MJ/m2)","Rainfall(mm)","Snowfall (cm)","Seasons","Holiday","Functioning Day"]
# this will contain our converters
colValues = {}

# first we load our data as strings so we can define the converters
data = np.array(np.genfromtxt('data/SeoulBikeData.csv', delimiter=',', usecols=(1,2,3,4,5,6,7,8,9,10,11,12,13), skip_header=1, dtype=str, encoding='utf-8'))  
print(data.shape[1])
# make a list of the unique values in each column of our data
for colIndex in range(data.shape[1]):
    colValues[colIndex] = np.unique(data[:, colIndex]).tolist()
    print(colIndex,"k ", colValues[colIndex])

# map values to their indices in the list of unique values
def converter(x, colIndex):
    return colValues[colIndex].index(x)
print(converter("Winter",10)) #3
print(len(columns))

13
0 k  ['0', '10', '100', '1000', '1001', '1002', '1003', '1004', '1005', '1006', '1007', '1008', '1009', '101', '1010', '1011', '1012', '1013', '1014', '1015', '1016', '1017', '1018', '1019', '102', '1020', '1021', '1022', '1023', '1024', '1025', '1026', '1027', '1028', '1029', '103', '1030', '1031', '1032', '1033', '1034', '1035', '1036', '1037', '1038', '1039', '104', '1040', '1041', '1042', '1044', '1045', '1046', '1047', '1049', '105', '1050', '1051', '1052', '1053', '1054', '1055', '1056', '1057', '1058', '1059', '106', '1060', '1061', '1062', '1063', '1064', '1065', '1066', '1067', '1068', '1069', '107', '1070', '1071', '1072', '1073', '1074', '1075', '1076', '1077', '1078', '1079', '108', '1080', '1082', '1083', '1084', '1085', '1086', '1087', '1088', '1089', '109', '1091', '1092', '1093', '1094', '1095', '1096', '1097', '1099', '11', '110', '1100', '1101', '1102', '1103', '1104', '1105', '1106', '1107', '1108', '1109', '111', '1110', '1111', '1112', '1113', '1114', '1115', '1

Now we actually load the data.

**check this**

In [47]:
# This dataset is the mazda subsample from https://www.kaggle.com/austinreese/craigslist-carstrucks-data after some cleanup
# data = np.array(np.genfromtxt("data/SeoulBikeData.csv", delimiter=',', usecols=(1,2,3,4,5,7,8,9,11))
data = np.array(np.genfromtxt('data/SeoulBikeData.csv', delimiter=",", usecols=(1,2,3,4,5,7,8,9,10,11,12,13), converters={1: lambda x: converter(x, 0),2: lambda x: converter(x, 1),3: lambda x: converter(x, 2), 4: lambda x: converter(x, 3), 5: lambda x: converter(x, 4),6: lambda x: converter(x, 5), 7: lambda x: converter(x,6), 8: lambda x: converter(x, 7),9: lambda x: converter(x, 8), 10: lambda x: converter(x, 9),11: lambda x: converter(x, 10),12: lambda x: converter(x, 11),13: lambda x: converter(x, 12),14: lambda x: converter(x, 13)}, skip_header=1, dtype=int, encoding='utf-8'))  

In [48]:
print(data)
print(data.shape)

[[1267    0  115 ...    3    1    1]
 [ 965    1  118 ...    3    1    1]
 [ 715   12  123 ...    3    1    1]
 ...
 [1831   14  289 ...    0    1    1]
 [1852   15  284 ...    0    1    1]
 [1713   16  182 ...    0    1    1]]
(8760, 12)


Let's get some summary statistics and do a **pairplot** so we can see what's going on.

In [49]:
def getSummaryStatistics(data):
    print("min, max, mean, std per variable")
    return pd.DataFrame([data.min(axis=0), data.max(axis=0), data.mean(axis=0), data.std(axis=0)])

def getShapeType(data):
    print("shape")
    return (data.shape, data.dtype)

print(getSummaryStatistics(data))
print(getShapeType(data))

min, max, mean, std per variable
            0          1           2          3          4           5   \
0     0.000000   0.000000    0.000000   0.000000   0.000000    0.000000   
1  2165.000000  23.000000  545.000000  89.000000  64.000000  555.000000   
2  1126.567237  11.500000  293.078082  49.243721  17.242466  316.257306   
3   658.302409   6.922187  137.441761  20.315105  10.331984  161.528699   

           6          7          8         9         10        11  
0    0.000000   0.000000   0.000000  0.000000  0.000000  0.000000  
1  344.000000  60.000000  50.000000  3.000000  1.000000  1.000000  
2   56.906050   1.101142   0.732877  1.495890  0.950685  0.966324  
3   86.853055   6.070123   4.110641  1.114345  0.216525  0.180393  
shape
((8760, 12), dtype('int64'))


In [50]:
df = pd.DataFrame(data, columns=columns)
seaborn.pairplot(df, y_vars = columns[0], x_vars = columns[1:])

plt.show()

ValueError: Shape of passed values is (8760, 12), indices imply (8760, 13)

It looks like which variables might be correlated with price?

Let's calculate *correlations* between price and the other variables. (Remind me what correlation values vary between?)

In [ ]:
print(len(columns))
for i in range(len(columns)): #correlation between the price and all the columns
    print(columns[i], np.corrcoef(data[:, 0], data[:, i], rowvar=True)[0,1])

13
Rented Bike Count 1.0
Hour -0.052522566840385185
Temperature(C) 0.06846982272066854
Humidity(%) 0.05595311446505868
Wind speed (m/s) -0.05891689595645904
Visibility (10m) -0.07965771340057491
Dew point temperature(C) -0.047535763183110216
Solar Radiation (MJ/m2) 0.004116910157714038
Rainfall(mm) 0.013776714164994949
Snowfall (cm) 0.11553169100548359
Seasons 0.03850509249480542
Holiday 0.31946935105319696


IndexError: index 12 is out of bounds for axis 1 with size 12

## Let's review regression

Regression allows us to:
* determine the *nature* of a relationship between one (or more!) independent variables and a dependent variable
* determine the *strength* of the relationship

Regression *fits* a function to a dataset.

## What kinds of functions can we fit? 

I want to predict price as a function of age and mileage! After all, an old car with low mileage may be worth more than a new car with high mileage. Actually, I probably want to include some of those other variables (features) too! (Which ones?)

It turns out I can do this using **multiple linear regression**. The function I will want to fit will be: $\hat{y} = c_0 + c_1*x_{1i} + c_2*x_{2i} + ... + c_M*x_{Mi}$ for $M$ variables, and I do this by minimizing the sum of the squares of the residuals $r_i = y_i - \hat{y_i}$.

In terms of matrix math, for $N$
 data points, $A$
 will just be a matrix of shape ($N, M+1$)
 (including the leading column of 1s) and $\vec{c}$ 
 will have shape ($M+1, 1$)
 (including $c_0$, the intercept) and $\vec{y}$ will have shape ($N, 1$) (as before).

Let's do it! 

### First, split our data

Let's split our data into **train** and **test**. Let's make sure and sort by time first, because we don't want to let the future predict the past.

In [ ]:
data = data[data[:, 1].argsort()]
print(getSummaryStatistics(data))
print(getShapeType(data))

(train, test) = np.split(data, [int(len(data) / 10 * 8)])
print(train.shape, test.shape)

min, max, mean, std per variable
            0          1           2          3          4           5   \
0     0.000000   0.000000    0.000000   0.000000   0.000000    0.000000   
1  2165.000000  23.000000  545.000000  89.000000  64.000000  555.000000   
2  1126.567237  11.500000  293.078082  49.243721  17.242466  316.257306   
3   658.302409   6.922187  137.441761  20.315105  10.331984  161.528699   

           6          7          8         9         10        11  
0    0.000000   0.000000   0.000000  0.000000  0.000000  0.000000  
1  344.000000  60.000000  50.000000  3.000000  1.000000  1.000000  
2   56.906050   1.101142   0.732877  1.495890  0.950685  0.966324  
3   86.853055   6.070123   4.110641  1.114345  0.216525  0.180393  
shape
((8760, 12), dtype('int64'))
(7008, 12) (1752, 12)


### Second, define an updated fit function that can handle multiple independent variables

In [53]:
def fit(data, independent, dependent):
    # These are our independent variable(s)
    x = data[np.ix_(np.arange(data.shape[0]), independent)]

    # We add a column of 1s for the intercept
    A = np.hstack((np.array([np.ones(x.shape[0])]).T, x))

    # This is the dependent variable 
    y = data[:, dependent]

    # This is the regression coefficients that were fit, plus some other results
    c, _, _, _ = sp_la.lstsq(A, y)
    print(c)
    return c

In [ ]:
print(columns)

NameError: name 'c' is not defined

And fit to our training data.

In [65]:
c = fit(train, [1,2,3,4,5,6,7,8,9,10,11], 0)
print([columns[x] for x in [1,2,3,4,5,6,7,8,9,10,11,12]])
print(c)

[ 3.13642722e+01 -2.08413434e+01  5.90524932e-01  3.04364002e+00
 -2.49574136e+00 -3.55691108e-01 -5.43621500e-01 -3.17012683e+00
 -3.17123856e+00  2.72252553e+01  9.10523049e+01  1.06066900e+03]
['Hour', 'Temperature(C)', 'Humidity(%)', 'Wind speed (m/s)', 'Visibility (10m)', 'Dew point temperature(C)', 'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Seasons', 'Holiday', 'Functioning Day']
[ 3.13642722e+01 -2.08413434e+01  5.90524932e-01  3.04364002e+00
 -2.49574136e+00 -3.55691108e-01 -5.43621500e-01 -3.17012683e+00
 -3.17123856e+00  2.72252553e+01  9.10523049e+01  1.06066900e+03]


Bad pipe message: %s [b'p\x02\x17_\xdb\x17\xf2\xdf\xace\x00D\xde`0_\x9dh \xcf.4\xa6\x03[Wp2/\x8e\x0f\x14\xab\x8f\xc6\x90\x04}\xe8\x1e\x9fj:\xa4H\xd7\xf3\xfb\xd2J\xf6\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \x99\xfc\x00\xe1\x84\xc1\xdfH\xcb8\xc7}\x8b\xab\xb3>\x98\xd9\xff\x1b\xfe\xe5']
Bad pipe message: %s [b'P\x9f\xdd\xb3X\xd7\xfb8#x`\xf9\x04+O),\xa1 \x96OET\xba\xc5h\x95\xd6\xa3\x08\x13\xb6\xe1\x91\x1c\x9c\x14 \x1aR\x8c\x9b\xff\x84\x10t}\xab\xb1\xe1>\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00

### Third, define an updated predict function that can handle multiple independent variables

In [62]:
def predict(data, independent, c):
    # These are our independent variable(s)
    x = data[np.ix_(np.arange(data.shape[0]), independent)]
 
    # We add a column of 1s for the intercept
    A = np.hstack((np.array([np.ones(x.shape[0])]).T, x))

    return np.dot(A, c)

And predict

In [63]:
yhat = predict(test, [1,2,3,4,5,6,7,8], c)

ValueError: shapes (1752,9) and (2,) not aligned: 9 (dim 1) != 2 (dim 0)

### Fourth, evaluate using $R^2$

In [59]:
# assume these are numpy arrays
def rsquared(y, yhat):
    if len(y) != len(yhat):
        print("Need y and yhat to be the same length!")
        return 0
    return 1 - (((y - yhat)**2).sum() / ((y - y.mean())**2).sum())

def plotxyyhat(x, y, yhat):
    plt.plot(x, y, 'o', label='data')
    plt.plot(x, yhat, label='least squares fit, $y = mx + b$')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend(framealpha=1, shadow=True)
    plt.grid(alpha=0.25)
    plt.show()

In [61]:
print(rsquared(test[:, 1], yhat))
# using only one independent variable to plot
plotxyyhat(test[:, 1], test[:, 0], yhat)


NameError: name 'yhat' is not defined